# Setting

In [34]:
import logging
import os
import pdb
import math
import glob
import random
import time
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.preprocessing import LabelEncoder

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

import torchvision.transforms as T

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.nn.parameter import Parameter

from tqdm import tqdm
import sys

import copy
from scipy import spatial
import csv

from perceiver import crop, patchify, get_patch_coords, ImageDataset, PerceiverBlock, Perceiver, CustomDataset

In [35]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)

# Load Model, DataLoader

In [36]:
root_dir = '/home/Minju/Perceiver/model/'
loader_dir = '/home/Minju/Perceiver/loader/'

batch_size = 32

In [37]:
input_models = []
valid_loaders = []

### Text Modality:

In [38]:
# for i in range (6):
#     text_model = torch.load(root_dir + f'text_model_{i+1}.pkl')
#     input_models.append(text_model)
#     print(f"Text model {i+1}번 불러오기 완료.")

#     with open(loader_dir+f'text_val_loader_{i+1}.pkl', 'rb') as f:
#         loaded_valid_dataset = pickle.load(f)
#     valid_loaders.append(loaded_valid_dataset)
#     print(f"Text val. loader {i}번 불러오기 완료.")

### Image Modality:

In [39]:
for i in range(6):
    img_model = torch.load(root_dir + f'image_model_{i+1}.pkl')
    input_models.append(img_model)
    print(f"Image model {i}번 불러오기 완료.")

    with open(loader_dir+f'image_val_loader_{i+1}.pkl', 'rb') as f:
        loaded_valid_dataset = pickle.load(f)

    valid_loader = DataLoader(loaded_valid_dataset, batch_size=batch_size, shuffle=False)
    valid_loaders.append(valid_loader)
    print(f"Image val. loader {i}번 불러오기 완료.")

/tmp/ipykernel_160219/520048891.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  img_model = torch.load(root_dir + f'image_model_{i+1}.pkl')


Image model 0번 불러오기 완료.
Image val. loader 0번 불러오기 완료.
Image model 1번 불러오기 완료.
Image val. loader 1번 불러오기 완료.
Image model 2번 불러오기 완료.
Image val. loader 2번 불러오기 완료.
Image model 3번 불러오기 완료.
Image val. loader 3번 불러오기 완료.
Image model 4번 불러오기 완료.
Image val. loader 4번 불러오기 완료.
Image model 5번 불러오기 완료.
Image val. loader 5번 불러오기 완료.


In [40]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## Define ModelDiff

In [41]:
num_classes = -1
lr = 0.1
batch_size = 32
val_batch_size = 100
workers = 24
weight_decay = 4e-5
dataset_name = ''
train_path = ''
val_path = ''
cuda = True
seed = 1
epochs = 160
restore_epoch = 0
save_folder = ''
load_folder = ''
one_shot_prune_perc = 0.5
mode = ''
logfile = ''
initial_from_task = ''

In [42]:
DATASETS = [
    'Opinion','Art & Design','Television',
    'Music','Travel','Real Estate',
    'Books','Theater','Health',
    'Sports','Science','Food',
    'Fashion & Style','Movies','Technology',
    'Dance', 'Media', 'Style'
]

In [43]:
epsilon = 0.1
max_iterations = 100

In [44]:
# target_id = 1

# 유사도검색

task_id 책정방식: \
0~5 : text modality \
6~11: image modality

## Approach 1: 특정 input data로 유사도 검증

In [45]:
def compute_ddv_cos(model1, model2, inputs):
    global outputs
    global outputs2
        
    with torch.no_grad():
        dists = []
        outputs = model1(torch.Tensor(inputs).cuda()).to('cpu').tolist()
        n_pairs = int(len(list(inputs)) / 2)
        for i in range(n_pairs):
            ya = outputs[i]
            yb = outputs[i + n_pairs]
            dist = spatial.distance.cosine(ya, yb)
            dists.append(dist)

        dists2 = []
        outputs2 = model2(torch.Tensor(inputs).cuda()).to('cpu').tolist()
        n_pairs2 = int(len(list(inputs)) / 2)
        for i in range(n_pairs2):
            ya = outputs2[i]
            yb = outputs2[i + n_pairs]
            dist = spatial.distance.cosine(ya, yb)
            dists2.append(dist)
    return np.array(dists), np.array(dists2)

In [46]:
def compute_ddv_euc(model1, model2, inputs):
    global outputs
    global outputs2
        
    with torch.no_grad():
        dists = []
        outputs = model1(torch.Tensor(inputs).cuda()).to('cpu').tolist()
        n_pairs = int(len(list(inputs)) / 2)
        for i in range(n_pairs):
            ya = outputs[i]
            yb = outputs[i + n_pairs]
            dist = spatial.distance.euclidean(ya, yb) # dist = spatial.distance.cosine(ya, yb)
            dists.append(dist)

        dists2 = []
        outputs2 = model2(torch.Tensor(inputs).cuda()).to('cpu').tolist()
        n_pairs2 = int(len(list(inputs)) / 2)
        for i in range(n_pairs2):
            ya = outputs2[i]
            yb = outputs2[i + n_pairs]
            dist = spatial.distance.euclidean(ya, yb) # dist = spatial.distance.cosine(ya, yb)
            dists2.append(dist)
    return np.array(dists), np.array(dists2)

In [47]:
##### compute_similarity #####
def compute_sim_cos(ddv1, ddv2):
    return spatial.distance.cosine(ddv1, ddv2)    

### modelDiff 

In [48]:
# ddvcc_list = []
# ddvec_list = []

# for task_id in range(6):
#     if task_id == target_id:
#         continue

#     # batch = next(iter(valid_loaders[target_id]))  
#     # print(batch)
#     # valid_loader = valid_loaders[target_id]  # valid_loader 가져오기
#     # data_iter = iter(valid_loader)  # 반복자 생성
#     # batch = next(data_iter)  # 첫 번째 배치 가져오기
    
#     # print(batch)  
    
#     # # input_ids를 numpy 또는 tensor로 변환
#     # inputs = batch['input_ids'].to(DEVICE)
#     if target_id >= 6:   # Text Modality
#         batch = next(iter(valid_loaders[target_id]))  
#         inputs = batch['input_ids'].to(DEVICE)
        
#     else:               # Image Modality
#         inputs = next(iter(valid_loaders[target_id]))[0]

#     ddv1, ddv2 = compute_ddv_cos(input_models[target_id], input_models[task_id], inputs)
#     ddv_distance = compute_sim_cos(ddv1, ddv2)
#     print('DDV cos-cos [%d => %d] %.5f'%(task_id, target_id, ddv_distance))
#     ddvcc_list.append(ddv_distance)

#     # DDV-EC
#     ddv1, ddv2 = compute_ddv_euc(input_models[target_id], input_models[task_id], inputs)
#     ddv_distance = compute_sim_cos(ddv1, ddv2)
#     print('DDV euc-cos [%d => %d] %.5f'%(task_id, target_id, ddv_distance))
#     ddvec_list.append(ddv_distance)

In [52]:
ddvcc_list = []
ddvec_list = []

best_cos_list = []
best_euc_list = []

for target_id in range(6):
    best_score_cos=0
    best_id_cos=0
    best_score_euc=0
    best_id_euc=0

    for task_id in range(6):
        if task_id == target_id:
            continue

        if target_id >= 6:   # Text Modality
            batch = next(iter(valid_loaders[target_id]))  
            inputs = batch['input_ids'].to(DEVICE)
        else:               # Image Modality
            inputs = next(iter(valid_loaders[target_id]))[0]

        ddv1, ddv2 = compute_ddv_cos(input_models[target_id], input_models[task_id], inputs)
        ddv_distance = compute_sim_cos(ddv1, ddv2)
        print('DDV cos-cos [%d => %d] %.5f'%(task_id, target_id, ddv_distance))
        ddvcc_list.append(ddv_distance)
        if ddv_distance >  best_score_cos :
            best_score_cos = ddv_distance
            best_id_cos = task_id


    # DDV-EC
        ddv1, ddv2 = compute_ddv_euc(input_models[target_id], input_models[task_id], inputs)
        ddv_distance = compute_sim_cos(ddv1, ddv2)
        print('DDV euc-cos [%d => %d] %.5f'%(task_id, target_id, ddv_distance))
        ddvec_list.append(ddv_distance)
        if ddv_distance >  best_score_euc :
            best_score_euc = ddv_distance
            best_id_euc = task_id
    best_cos_list.append(best_id_cos)
    best_euc_list.append(best_id_euc)

for i in range(6):
    print(f'model {i}의 cos 기준 backbone: {best_cos_list[i]}')
    print(f'model {i}의 euc 기준 backbone: {best_euc_list[i]}')

DDV cos-cos [1 => 0] 0.33892
DDV euc-cos [1 => 0] 0.22612
DDV cos-cos [2 => 0] 0.65785
DDV euc-cos [2 => 0] 0.33747
DDV cos-cos [3 => 0] 0.16940
DDV euc-cos [3 => 0] 0.08735
DDV cos-cos [4 => 0] 0.08639
DDV euc-cos [4 => 0] 0.07176
DDV cos-cos [5 => 0] 0.66696
DDV euc-cos [5 => 0] 0.34416
DDV cos-cos [0 => 1] 0.38898
DDV euc-cos [0 => 1] 0.18607
DDV cos-cos [2 => 1] 0.30110
DDV euc-cos [2 => 1] 0.12600
DDV cos-cos [3 => 1] 0.28613
DDV euc-cos [3 => 1] 0.09757
DDV cos-cos [4 => 1] 0.58688
DDV euc-cos [4 => 1] 0.23617
DDV cos-cos [5 => 1] 0.40869
DDV euc-cos [5 => 1] 0.12332
DDV cos-cos [0 => 2] 0.40153
DDV euc-cos [0 => 2] 0.20917
DDV cos-cos [1 => 2] 0.57333
DDV euc-cos [1 => 2] 0.21514
DDV cos-cos [3 => 2] 0.46248
DDV euc-cos [3 => 2] 0.18565
DDV cos-cos [4 => 2] 0.80110
DDV euc-cos [4 => 2] 0.43881
DDV cos-cos [5 => 2] 0.45114
DDV euc-cos [5 => 2] 0.18134
DDV cos-cos [0 => 3] 0.05162
DDV euc-cos [0 => 3] 0.02070
DDV cos-cos [1 => 3] 0.43202
DDV euc-cos [1 => 3] 0.14401
DDV cos-cos [2

## Approach 2: latent vector로 유사도 검증